In [1]:
using LinearAlgebra
using QuantumOptics
using WignerSymbols
using Plots
plotly()
using Printf 
import QuantumOptics.steadystate as steady

In [39]:
nucI=3//2
grJ=1//2
exJ=1//2

exFmax=nucI+exJ
exFmin=abs(nucI-exJ)
grFmax=2  #nucI+grJ
grFmin=2  #abs(nucI-grJ)

#grStates=Dict([((2,2),1),((2,1),2),((2,0),3),((2,-1),4),((2,-2),5),((1,1),6),((1,0),7),((1,-1),8)])
#exStates=Dict([((2,2),9),((2,1),10),((2,0),11),((2,-1),12),((2,-2),13),((1,1),14),((1,0),15),((1,-1),16)])
#rbD1=NLevelBasis(17)  # One extra state for fly-through relaxation
grStates=Dict([((2,2),1),((2,1),2),((2,0),3),((2,-1),4),((2,-2),5)])
exStates=Dict([((2,2),6),((2,1),7),((2,0),8),((2,-1),9),((2,-2),10),((1,1),11),((1,0),12),((1,-1),13)])
rbD1=NLevelBasis(14)  # One extra state for fly-through relaxation


function sigma(theBasis,gr,ex,F1,F2,q,mF)
    transition(theBasis,gr[(F2,mF),ex(F2,mF+q)]) 
end

function dipole(J1,J2,nucI,F1,F2,q,mF)
    wigner3j(F1,1,F2,mF,q,-mF-q)*wigner6j(J1,J2,1,F2,F1,nucI)*(-1)^(mF+J1+nucI)*sqrt((2F1+1)*(2*F2+1)*(2*J1+1))
end 


σ=[] #Array{Operator}()
dip=[]

#F1vals=[1,2]
F1vals=[2]
F2vals=[1,2]
qvals=[-1,1]

n2Fm=Dict{Int64,Tuple}()
Fm2n=Dict{Tuple,Int64}()
myIndex=1

for F=grFmax:-1:grFmin
    for m=F:-1:-F
        get!(n2Fm,myIndex,("ground",(F,m)))
        get!(Fm2n,("ground", (F,m)),myIndex)
        myIndex+=1
    end
end

for F=exFmax:-1:exFmin
    for m=F:-1:-F
        get!(n2Fm,myIndex,("excited", (F,m)))
        get!(Fm2n,("excited", (F,m)),myIndex)
        myIndex+=1
    end
end

for i in keys(n2Fm)
    if n2Fm[i][1]=="ground"
        F1=n2Fm[i][2][1]
        m1=n2Fm[i][2][2]
        for F2 = exFmin:exFmax
            if (newN=get(Fm2n,("excited",(F2,m1+1)),0)) > 0
                q=+1
 #               println("σ+: i = ", i, "  newN = ", newN, "  F1 = ", F1, "  m1 = ", m1, "  F2 = ", F2, "  m1+1 = ", (m1+1), " 3j = ", wigner3j(F1,1,F2,m1,q,-m1-q), " 6j = ", wigner6j(exJ,grJ,1,F2,F1,nucI))
                push!(σ,transition(rbD1,i,newN))
                push!(dip,dipole(grJ,exJ,nucI,F1,F2,q,m1))
            end
            if (newN=get(Fm2n,("excited",(F2,m1-1)),0)) > 0
                q=-1
#                println("σ-: i = ", i, "  newN = ", newN, "  F1 = ", F1, "  m1 = ", m1, "  F2 = ", F2, "  m1-1 = ", (m1-1), " 3j = ", wigner3j(F1,1,F2,m1,q,-m1-q), " 6j = ", wigner6j(exJ,grJ,1,F2,F1,nucI))
                push!(σ,transition(rbD1,i,newN))
                push!(dip,dipole(grJ,exJ,nucI,F1,F2,q,m1))
            end
        end
    end
end



In [43]:
hfsEx=814.5
hfsGr=6834.682610904290
grF1=-0.625*hfsGr
grF2=0.375*hfsGr
exF1=-0.625*hfsEx
exF2=0.375*hfsEx
Δ=1.0
hfsE=Dict(  ("ground",(1,1))=>hfsGr,
            ("ground",(1,0))=>hfsGr, 
            ("ground",(1,-1))=>hfsGr,
            ("ground",(2,2))=>0, 
            ("ground",(2,1))=>0,
            ("ground",(2,0))=>0,
            ("ground",(2,-1))=>0,
            ("ground",(2,-2))=>0,
            ("excited",(1,1))=>-Δ+hfsEx,
            ("excited",(1,0))=>-Δ+hfsEx, 
            ("excited",(1,-1))=>-Δ+hfsEx,
            ("excited",(2,2))=>-Δ, 
            ("excited",(2,1))=>-Δ,
            ("excited",(2,0))=>-Δ,
            ("excited",(2,-1))=>-Δ,
            ("excited",(2,-2))=>-Δ,
            )
ωLGr1=-7000  # MHz/Tesla   Larmor frequency
ωLGr2=7000 
ωLEx1=-2300  # MHz/Tesla   Larmor frequency
ωLEx2=2300   
larmor=Dict(("ground",(1,1))=> 1.0*ωLGr1,
            ("ground",(1,0))=> 0*ωLGr1, 
            ("ground",(1,-1))=> -1.0*ωLGr1,
            ("ground",(2,2))=> 2.0*ωLGr2, 
            ("ground",(2,1))=> 1.0*ωLGr2,
            ("ground",(2,0))=> 0*ωLGr2,
            ("ground",(2,-1))=> -1.0*ωLGr2,
            ("ground",(2,-2))=> -2.0*ωLGr2,
            ("excited",(1,1))=> 1.0*ωLEx1,
            ("excited",(1,0))=> 0*ωLEx1, 
            ("excited",(1,-1))=> -1.0*ωLEx1,
            ("excited",(2,2))=> 2.0*ωLEx2, 
            ("excited",(2,1))=> 1.0*ωLEx2,
            ("excited",(2,0))=> 0*ωLEx2,
            ("excited",(2,-1))=> -1.0*ωLEx2,
            ("excited",(2,-2))=> -2.0*ωLEx2,
            )
Ω=20.0       # Rabi frequency
Γ=7.0       # Natual linewidth
γ=0.5       # Transit relaxation
flyThrough = 14 #17 # Level for transit (fly-through) relaxation
nGround    = 8  # number of levels in the ground state 

#gamma=Dict(("ground",(1,1))=>γ,
#           ("ground",(1,0))=>γ, 
 #           ("ground",(1,-1))=>γ,
 #           ("ground",(2,2))=>γ, 
 #           ("ground",(2,1))=>γ,
 #           ("ground",(2,0))=>γ,
 #           ("ground",(2,-1))=>γ,
 #           ("ground",(2,-2))=>γ,
 #           ("excited",(1,1))=>γ+Γ,
 #           ("excited",(1,0))=>γ+Γ, 
 #           ("excited",(1,-1))=>γ+Γ,
 #           ("excited",(2,2))=>γ+Γ, 
 #           ("excited",(2,1))=>γ+Γ,
 #           ("excited",(2,0))=>γ+Γ,
 #           ("excited",(2,-1))=>γ+Γ,
 #           ("excited",(2,-2))=>γ+Γ,
 #           )

#Δ=100       # Detuning from D1 transition at 794.978851156 nm (377.107464380 THz)

proj=[]
detune=[]
ωL=[]
H=identityoperator(σ[1])*0
#J=identityoperator(σ[1])*0
J=[]
for i=1:length(n2Fm)
    push!(proj,transition(rbD1,i,i))
    push!(ωL,larmor[n2Fm[i]])
    push!(detune,hfsE[n2Fm[i]])
 #   H += ( detune[i] + ωL[i]*0.001 )*proj[i] # diagonal part of Hamiltonian
    push!(J,sqrt(γ)*transition(rbD1,i,flyThrough))                            # Jump for fly-through relaxation out
    if n2Fm[i][1] == "ground"
        push!(J,sqrt(γ/nGround)*transition(rbD1,flyThrough,i))                    # Jump for fly-through relaxation in 
    end
    for j=1:length(n2Fm)
        if n2Fm[i][1]=="excited" && n2Fm[j][1]=="ground"
             push!(J,sqrt(Γ/nGround)*transition(rbD1,i,j))
        end
    end
end

#for i=1:length(σ)
#    H+= Ω*dip[i]*(σ[i]+dagger(σ[i])) # off-diagonal part of Hamiltonian
#end

# Initial state
ρ₀ = (dm(nlevelstate(rbD1,4))
      +dm(nlevelstate(rbD1,5))
      +dm(nlevelstate(rbD1,6))
      +dm(nlevelstate(rbD1,7))
      +dm(nlevelstate(rbD1,8))
      +5*dm(nlevelstate(rbD1,flyThrough))
      )/10



Operator(dim=14x14)
  basis: NLevel(N=14)
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.

In [46]:
Bmin=-5.0e-6
Bmax=5.0e-6
Bstep=1.0e-7
ΩR=1.0       # Rabi frequency
Γ=1.0       # Natual linewidth
γ=0.05       # Transit relaxation
Bvalues=[]
absorb=[]
rotation=[]
popEx=[]
popGr=[]
ρ₀ =(
    dm(nlevelstate(rbD1,1))
    +dm(nlevelstate(rbD1,2))
    +dm(nlevelstate(rbD1,3))
    +dm(nlevelstate(rbD1,4))
    +dm(nlevelstate(rbD1,5))
    +5*dm(nlevelstate(rbD1,flyThrough))
    )/10
for B=Bmin:Bstep:Bmax
    H=identityoperator(σ[1])*0
    for i=1:length(n2Fm)
#        push!(ωL,larmor[n2Fm[i]]*B)
#        push!(detune,hfsE[n2Fm[i]]-Δ)
        H += (  detune[i] + ωL[i]*B )*proj[i]
#        H += (  ωL[i]*B )*proj[i]
    end
    for i=1:length(σ)
        H+= ΩR*dip[i]*(σ[i]+dagger(σ[i]))   # Hamiltonian
    end
        # Initial state
#    ρ₀ =(
#        dm(nlevelstate(rbD1,4))
#        +dm(nlevelstate(rbD1,5))
#        +dm(nlevelstate(rbD1,6))
#        +dm(nlevelstate(rbD1,7))
#        +dm(nlevelstate(rbD1,8))
#        +5*dm(nlevelstate(rbD1,flyThrough))
#        )/10
#    t_out, ρ_master = steady.master(H,J;rho0=ρ₀,maxiters=1e6)
    ρ_eig=steady.eigenvector(DenseOperator(H),J)
    obs1=3*(Γ/ΩR)*real(expect(σ[1],ρ_eig) #ρ_master[2])
                        +expect(σ[2],ρ_eig) #ρ_master[2])
                        +expect(σ[3],ρ_eig) #ρ_master[2])
                        +expect(σ[4],ρ_eig) #ρ_master[2])
                        +expect(σ[5],ρ_eig) #ρ_master[2])
                        +expect(σ[6],ρ_eig) #ρ_master[2])
                        +expect(σ[7],ρ_eig) #ρ_master[2])
                        + -expect(σ[8],ρ_eig) #ρ_master[2])
                        + - expect(σ[9],ρ_eig) #ρ_master[2])
                        + -expect(σ[10],ρ_eig) #ρ_master[2])
                        +  -expect(σ[11],ρ_eig) #ρ_master[2])
                        + -expect(σ[12],ρ_eig) #ρ_master[2])
                        -expect(σ[13],ρ_eig) #ρ_master[2])
                        -expect(σ[14],ρ_eig) #ρ_master[2])
 #                       -expect(σ[15],ρ_master[2])
 #                       -expect(σ[16],ρ_master[2])
 #                       -expect(σ[17],ρ_master[2])
 #                       -expect(σ[18],ρ_master[2])
 #                       -expect(σ[19],ρ_master[2])
 #                       -expect(σ[20],ρ_master[2])
 #                       -expect(σ[21],ρ_master[2])
 #                       -expect(σ[22],ρ_master[2])
 #                       -expect(σ[23],ρ_master[2])
 #                       -expect(σ[24],ρ_master[2])
                        )
    obs2=3*(Γ/ΩR)*real(expect(σ[1],ρ_eig) #ρ_master[2])
                        +expect(σ[2],ρ_eig) #ρ_master[2])
                        +expect(σ[3],ρ_eig) #ρ_master[2])
                        +expect(σ[4],ρ_eig) #ρ_master[2])
                        +expect(σ[5],ρ_eig) #ρ_master[2])
                        +expect(σ[6],ρ_eig) #ρ_master[2])
                        +expect(σ[7],ρ_eig) #ρ_master[2])
                        +expect(σ[8],ρ_eig) #ρ_master[2])
                        +expect(σ[9],ρ_eig) #ρ_master[2])
                        +expect(σ[10],ρ_eig) #ρ_master[2])
                        +expect(σ[11],ρ_eig) #ρ_master[2])
                        +expect(σ[12],ρ_eig) #ρ_master[2])
                        +expect(σ[13],ρ_eig) #ρ_master[2])
                        +expect(σ[14],ρ_eig) #ρ_master[2])
 #                       +expect(σ[15],ρ_master[2])
 #                       +expect(σ[16],ρ_master[2])
 #                       +expect(σ[17],ρ_master[2])
 #                       +expect(σ[18],ρ_master[2])
 #                       +expect(σ[19],ρ_master[2])
 #                       +expect(σ[20],ρ_master[2])
 #                       +expect(σ[21],ρ_master[2])
 #                       +expect(σ[22],ρ_master[2])
 #                       +expect(σ[23],ρ_master[2])
 #                       +expect(σ[24],ρ_master[2])
                        )
    obs3=real(
        expect(proj[9],ρ_eig) #ρ_master[2])
        +expect(proj[10],ρ_eig) #ρ_master[2])
        +expect(proj[11],ρ_eig) #ρ_master[2])
        +expect(proj[12],ρ_eig) #ρ_master[2])
        +expect(proj[13],ρ_eig) #ρ_master[2])
        +expect(proj[6],ρ_eig) #ρ_master[2])
        +expect(proj[7],ρ_eig) #ρ_master[2])
        +expect(proj[8],ρ_eig) #ρ_master[2])
        )
        obs4=real(
            expect(proj[1],ρ_eig) #ρ_master[2])
            +expect(proj[2],ρ_eig) #ρ_master[2])
            +expect(proj[3],ρ_eig) #ρ_master[2])
            +expect(proj[4],ρ_eig) #ρ_master[2])
            +expect(proj[5],ρ_eig) #ρ_master[2])
#            +expect(proj[6],ρ_master[2])
#            +expect(proj[7],ρ_master[2])
#            +expect(proj[8],ρ_master[2])
            )
    push!(Bvalues,B)
    push!(absorb,obs1)
    push!(rotation,obs2)
    push!(popEx,obs3)
    push!(popGr,obs4)
end

In [47]:
plot1=Plots.plot(Bvalues,popEx)
plot2=Plots.plot(Bvalues,popGr)
plot3=Plots.plot(Bvalues,absorb)
plot4=Plots.plot(Bvalues,rotation)
Plots.plot(plot1, plot2, plot3, plot4, layout = (2, 2), legend = false)

<!DOCTYPE html>
 
 
 Plots.jl